## Simulation d'une situation réel avec EnsembleSet

In [ ]:
import IPython.display as ipd
from IPython.display import display
import pickle
import librosa
import os
import matplotlib.pyplot as plt
import itertools
import operator
import soundfile as sf
import scipy as sp
import numpy as np
from numpy import typing
import pyroomacoustics as pra
from mir_eval.separation import bss_eval_sources

path_in = "./data/prise_de_son/"
save_path = "./test/prise_de_son_perf/"

In [ ]:
from src import performance, data_processing, fast_nmf

### Chargmenent de la data et preprocess

In [ ]:
# load the data
files_in, files_tilte = data_processing.get_files(path_in, ".wav")

In [ ]:
nb_mics = 2  # number of microphones
audio_length = 10  # seconds
samplerate = 44100  # sampling rate

mics_signals = np.zeros((nb_mics, audio_length * samplerate), dtype=np.float32)
microphone_names = ["microphone_1", "microphone_2"]

# extract data
for i, file in enumerate(files_in):
    data, samplerate = librosa.load(file, sr=None, mono=False)
    mics_signals[i, :] = np.mean(
        data[:, : audio_length * samplerate], axis=0
    )  # average on channels and stack signals

In [ ]:
print(f"Microphone: {microphone_names[0]}")
ipd.Audio(mics_signals[0], rate=samplerate)

In [ ]:
print(f"Microphone: {microphone_names[1]}")
ipd.Audio(mics_signals[1], rate=samplerate)

### Définition des variables globales et création des dossiers de visualisations

In [ ]:
SAVE_FIG = (
    False  # Si True, on sauvegarde toutes les figures dans les sous-dossiers de 'test'
)
SAVE_AUDIO = False  # Si True, on sauvegarde tous les fichiers audio dans le sous-dossiers 'audio' de 'test'
SAVE_PERF = (
    False  # Si True, on sauvegarde toutes les figures dans les sous-dossiers de 'test'
)
TYPE = "stft"

In [ ]:
# Permet de créé les dossiers de sauvegarde si les variables permettant la sauvegarde
# sont à True

try:
    os.mkdir("./test")

except OSError as error:
    pass


if SAVE_FIG:
    try:
        os.mkdir(save_path)

        try:
            os.mkdir(save_path + "activation")
            os.mkdir(save_path + "base")
            os.mkdir(save_path + "spectro")
            os.mkdir(save_path + "mix")
        except OSError as error:
            pass

    except OSError as error:
        pass

if SAVE_AUDIO:
    try:
        os.mkdir(save_path + "audios")
        os.mkdir(save_path + "audios/separation")

        os.mkdir(save_path + "audios/separation/micro_1")
        os.mkdir(save_path + "audios/separation/micro_2")

    except OSError as error:
        pass

In [ ]:
L = 2048  # Taille de la fenêtre
hop = 512  # Pas de la fenêtre

# Transformations des signaux audio en STFT_multichannel en prennant en compte la room et l'emplacement des micros et sources
X = data_processing.spectrogram_from_mics_signal(
    mics_signals,
    microphone_names,
    rate=samplerate,
    L=L,
    hop=hop,
    type=TYPE,
    display_audio=True,
    display_spectrogram=False,
)

#### Définitions des W (Dictionnaires)

In [ ]:
dico_path = "./data/dictionnaire/piano/"
pitch_min = 21
pitch_max = 109

W_piano, notes_piano, notes_piano_psd, freqs = data_processing.compute_W(
    dico_path, L=L, pitch_min=21, pitch_max=109
)

In [ ]:
# visualisation du dictionnaire w du piano

fig = plt.figure()
fig.set_size_inches(8, 4)

plt.imshow(W_piano, aspect="auto")
plt.gca().invert_yaxis()
plt.tight_layout()

plt.title("Dictionnaire W du piano")
plt.show()

In [ ]:
# visualisation d'un psd d'un note du piano

fig = plt.figure()
fig.set_size_inches(8, 4)

plt.semilogy(freqs * 22050, notes_piano_psd[27])
plt.xlabel("frequency [Hz]")
plt.ylabel("PSD [V**2/Hz]")

plt.tight_layout()
plt.title("PSD de la note 27 du piano")
plt.show()

### Fast MNMF pour la séparation

In [ ]:
# Parametres de l'algorithme
n_basis = 32
n_iter = 100

print("Running fastmnmf2 with Louis custom algorithm")
Y, W_NFK, H_NKT, g_NM, Q_FMM, Qx_FTM, X_tilde_FTM, Y_tilde_FTM = fast_nmf.fast_MNMF2(
    X.transpose(1, 0, 2),
    n_iter=n_iter,
    n_microphones=mics_signals.shape[0],
    n_sources=2,
    n_time_frames=X.shape[0],
    n_freq_bins=X.shape[1],
    n_basis=n_basis,
    algo="IP",
    mic_index=None,
)
Y = Y.transpose(0, 3, 2, 1)

### Un peu de visualisation

In [ ]:
plt.specgram(mics_signals[0], NFFT=1024, Fs=samplerate)
plt.title("Signal micro 1")

In [ ]:
# Correspond à la séparation de source, micro_plot_sep
mic_plot_sep = 0

fig = plt.figure()
fig.set_size_inches(10, 6)

plt.subplot(2, 2, 1)
plt.specgram(y[mic_plot_sep][0, :], NFFT=1024, Fs=samplerate)
plt.title("Source 0 (séparé)")

plt.subplot(2, 2, 2)
plt.specgram(y[mic_plot_sep][1, :], NFFT=1024, Fs=samplerate)
plt.title("Source 1 (séparé)")

plt.subplot(2, 2, 3)
plt.specgram(y[mic_plot_sep][2, :], NFFT=1024, Fs=samplerate)
plt.title("Source 2 (séparé)")

plt.subplot(2, 2, 4)
plt.specgram(y[mic_plot_sep][3, :], NFFT=1024, Fs=samplerate)
plt.title("Source 3 (séparé)")

plt.tight_layout()

fig.tight_layout(pad=2.5)
fig.suptitle(
    "spectro_source01_micro_"
    + microphone_names[mic_plot_sep]
    + "_n_basis_"
    + str(n_basis)
    + "_n_fft_"
    + str(L)
)

if SAVE_FIG:
    fig.savefig(
        save_path
        + "spectro/"
        + "spectro_source0123_micro_"
        + microphone_names[mic_plot_sep]
        + "_n_basis_"
        + str(n_basis)
        + "_n_fft_"
        + str(L)
        + ".pdf"
    )

#### Ecoute maintenant 

In [ ]:
if TYPE == "stft":
    # STFT parameters
    win_a = pra.hamming(L)
    win_s = pra.transform.stft.compute_synthesis_window(win_a, hop)

    y = []
    for i in range(len(Y)):
        signal_ = pra.transform.stft.synthesis(Y[i], L, hop, win=win_s)
        signal_ = signal_[L - hop :, :].T
        y.append(signal_)

elif TYPE == "cqt":
    y = []
    for i in range(len(Y)):
        signal_ = librosa.icqt(
            Y[i].transpose(2, 1, 0),
            sr=rate,
            hop_length=hop,
            fmin=None,
            bins_per_octave=12,
            tuning=0.0,
            filter_scale=1,
            norm=1,
            sparsity=0.01,
            window="hann",
            scale=True,
            length=None,
            res_type="fft",
            dtype=None,
        )
        y.append(signal_)

# shape of y = (n_mics, n_sources, n_samples)
y = np.array(y)

##### Pas de séparation

In [ ]:
print("Listening of Audio at each microphones without separation")

for micro_n in range(mics_signals.shape[0]):
    display(ipd.Audio(mics_signals[micro_n, :], rate=44100))
    if SAVE_AUDIO:
        sf.write(
            save_path
            + "audios/no_separation/micro_"
            + microphone_names[micro_n]
            + "/"
            + "globals_micro_"
            + microphone_names[micro_n]
            + ".wav",
            44100,
            mics_signals[micro_n],
        )

##### Séparation des sources

In [ ]:
def ecoute_separation_micro(mic, y, save):
    """Fonction permettant d'écouter les audios séparés pour un micro donné et de les sauvegarder si save=True

    Args:
        mic (int): index du microphone dont on veut écouter la séparation
        y (array): array contenant les signaux audios séparés pour chaque microphones
        save (boolean): True si on veut sauvegarder les audios séparés, False sinon
    """

    for source_n in range(len(y[0])):
        print(
            "Listening of Audio at microphone "
            + str(mic)
            + " for source "
            + str(source_n)
        )
        display(ipd.Audio(y[mic][source_n], rate=44100))

In [ ]:
print("Listening of Audios at microphones 0 with separation")
ecoute_separation_micro(0, y, SAVE_AUDIO)

In [ ]:
print("Listening of Audios at microphones 1 (couple) with separation")
ecoute_separation_micro(1, y, SAVE_AUDIO)

### Visualisation des matrices intermédiaires

#### Représentation of g, and G (covariance et matrice spatiale)

In [ ]:
plt.imshow(g_NM[:, :], cmap="inferno", aspect="auto")
plt.title("g_NM")
plt.tight_layout()

if SAVE_FIG:
    plt.savefig(
        save_path
        + "mix/"
        + "g_mn"
        + "_n_basis_"
        + str(n_basis)
        + "_n_fft_"
        + str(L)
        + ".pdf"
    )

In [ ]:
def G_mix(Q_FMM, g_NM):
    g_NMM = []
    for n_source in range(g_NM.shape[0]):
        g_NMM.append(np.diag(g_NM[n_source]))

    g_NMM = np.array(g_NMM)

    G_NF = np.einsum(
        "fij, nij, ijf -> ni",
        np.linalg.inv(Q_FMM),
        g_NMM,
        np.linalg.inv(Q_FMM).conj().T,
    )
    return G_NF

In [ ]:
# G_NF
G_NF = G_mix(Q_FMM, g_NM)

In [ ]:
plt.imshow(np.log(np.abs(G_NF)), cmap="inferno", aspect="auto")
plt.colorbar()
plt.title("G_NF_n_basis_" + str(n_basis) + "_n_fft_" + str(L))
plt.tight_layout()

if SAVE_FIG:
    plt.savefig(
        save_path
        + "mix/"
        + "G_NF"
        + "_n_basis_"
        + str(n_basis)
        + "_n_fft_"
        + str(L)
        + ".pdf"
    )

#### Représentation de W

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 6)

plt.subplot(2, 2, 1)
plt.imshow(np.log(W_NFK[0, :, :]), cmap="inferno", aspect="auto")
plt.gca().invert_yaxis()
plt.title("Matrice de base W0")

plt.subplot(2, 2, 2)
plt.imshow(np.log(W_NFK[1, :, :]), cmap="inferno", aspect="auto")
plt.gca().invert_yaxis()
plt.title("Matrice de base W1")

plt.subplot(2, 2, 3)
plt.imshow(np.log(W_NFK[2, :, :]), cmap="inferno", aspect="auto")
plt.gca().invert_yaxis()
plt.title("Matrice de base W2")

plt.subplot(2, 2, 4)
plt.imshow(np.log(W_NFK[3, :, :]), cmap="inferno", aspect="auto")
plt.gca().invert_yaxis()
plt.title("Matrice de base W3")

plt.tight_layout()

fig.tight_layout(pad=2.5)
fig.suptitle("W_NFK_n_basis_" + str(n_basis) + "_n_fft_" + str(L))

if SAVE_FIG:
    fig.savefig(
        save_path
        + "base/"
        + "W_NFK"
        + "_n_basis_"
        + str(n_basis)
        + "_n_fft_"
        + str(L)
        + ".pdf"
    )

#### Représentation des activations

In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 6)

plt.subplot(2, 2, 1)
plt.imshow(H_NKT[0, :, :], cmap="inferno", aspect="auto")
plt.gca().invert_yaxis()
plt.title("Matrice de base H0")

plt.subplot(2, 2, 2)
plt.imshow(H_NKT[1, :, :], cmap="inferno", aspect="auto")
plt.gca().invert_yaxis()
plt.title("Matrice de base H1")

plt.subplot(2, 2, 3)
plt.imshow(H_NKT[2, :, :], cmap="inferno", aspect="auto")
plt.gca().invert_yaxis()
plt.title("Matrice de base H2")

plt.subplot(2, 2, 4)
plt.imshow(H_NKT[3, :, :], cmap="inferno", aspect="auto")
plt.gca().invert_yaxis()
plt.title("Matrice de base H3")

plt.tight_layout()

fig.tight_layout(pad=2.5)
fig.suptitle("H_NKT_n_basis_" + str(n_basis) + "_n_fft_" + str(L))

if SAVE_FIG:
    fig.savefig(
        save_path
        + "activation/"
        + "H_NKT"
        + "_n_basis_"
        + str(n_basis)
        + "_n_fft_"
        + str(L)
        + ".pdf"
    )